In [2]:
import os
import io
import numpy as np
from numpy.random import rand
import torch
from torchsummary import summary
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import mpl_toolkits.mplot3d
from minetest.adapter import *
from colab.conv_gan_gp import *

In [ ]:
# define model parameters
output_shape = 16
noise_dim = 200
feature_dim = 64

generator = ConvGenerator(noise_dim, feature_dim)
discriminator = ConvDiscriminator(feature_dim, output_shape)

G_PATH = os.getcwd() + '/models/states/02-10-2022-10-36-17_G.pth'
D_PATH = os.getcwd() + '/models/states/ConvGAN18-09-2022-22-09-38_D.pth'
generator.load_state_dict(torch.load(G_PATH, map_location=torch.device("cpu")), strict=False)
discriminator.load_state_dict(torch.load(D_PATH, map_location=torch.device("cpu")), strict=False)

summary(generator, (1, noise_dim))
summary(discriminator, (1, output_shape, output_shape, output_shape))

start_training(
    generator,
    discriminator,
    data_path="segmentation_3d_data.pkl",
    learning_rate=0.0002,
    n_epochs=200,
    t="log_d",
    batch_size=64,
    noise_dim=noise_dim
)

<All keys matched successfully>

In [ ]:
with open(os.getcwd() + "/dataset/output/structures.pkl",'rb') as f:
    data = pickle.load(f)
mta = MinetestAdapter()
mta.connect()

In [ ]:
# value used for up-scaling data
MAX_ID = 32.0

# number of outputs to generate
LATENT_VECTORS = 12

generator.eval()
x = torch.Tensor(LATENT_VECTORS, noise_dim).normal_(0.0, 1.0)
s = np.fix(generator(x).detach().numpy().squeeze() * MAX_ID)
print(s.shape)

for i in range(LATENT_VECTORS):
    mta.build(s[i], (i*20-40, 0, 0))

(12, 16, 16, 16)


In [ ]:
samples = np.random.rand(4, 16, 16, 16)

plt.figure(figsize=(8, 4))
gs = gridspec.GridSpec(1, 4)
gs.update(wspace=0.05, hspace=0.05)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
for i, sample in enumerate(samples):
    x, y, z = sample.nonzero()
    ax = plt.subplot(gs[i], projection='3d')
    ax.scatter3D(x, y, z, zdir='z', c=sample[x,y,z], cmap='jet', marker="h", alpha=0.7, linewidth=0.)
    ax.axis('off')
# plt.savefig(os.getcwd() + 'sample_@epoch{}.png'.format(str(1)))
plt.show()
plt.close()

In [ ]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [ ]:
with open(os.getcwd() + '/models/logs/30-09-2022-17-35-19_log.pkl','rb') as f:
    stats = CPU_Unpickler(f).load()

x = list(range(len(stats['dl'])))
fig, ax = plt.subplots()
fig.set_figwidth(8)
ax.plot(x, [val * 100 for val in stats['dl']], label='discriminator loss * 100')
ax.plot(x, [val * 10 for val in stats['al']], label='adversarial loss * 10')
ax.plot(x, stats['ra'], label='real accuracy')
ax.plot(x, stats['fa'], label='fake accuracy')
ax.plot(x, stats['ma'], label='mean accuracy')
plt.ylim(0, 1.1)
plt.xlim(0, 100)
plt.legend(loc='lower left', bbox_to_anchor=(0.7, 0.2))

ax.set(xlabel='epoch', title='ConvGAN Training - Binary Cross Entropy')
ax.grid()

fig.savefig("conv_gan_BCE_training.png")
plt.show()

In [ ]:
with open(os.getcwd() + '/models/logs/02-10-2022-10-36-17_log.pkl','rb') as f:
    stats = CPU_Unpickler(f).load()

x = list(range(len(stats['dl'])))
fig, ax = plt.subplots()
fig.set_figwidth(16)
ax.plot(x, [val * 10 for val in stats['dl']], label='discriminator loss * 100')
ax.plot(x, [val * -100 for val in stats['al']], label='adversarial loss * -100')
ax.plot(x, stats['ra'], label='real accuracy')
ax.plot(x, stats['fa'], label='fake accuracy')
ax.plot(x, stats['ma'], label='mean accuracy')
plt.ylim(0, 1.1)
plt.xlim(0, 200)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)

ax.set(xlabel='epoch', title='ConvGAN Training - Wasserstein Gradient Penalty')
ax.grid()

fig.savefig("conv_gan_WGP_training.png")
plt.show()

In [ ]:
# label flip and noisify algorithm
FLIP_CHANCE = 1.5
NOISE_LEVEL = 0.4
OFFSET = 0.8
flips = np.floor(rand(3) * FLIP_CHANCE) * OFFSET
real = rand(3).astype(np.float32) * NOISE_LEVEL + OFFSET
fake = rand(3).astype(np.float32) * NOISE_LEVEL
real -= flips
fake += flips

print(real)
print(fake)
print(flips)


[0.23538752 0.9114854  0.9910328 ]
[1.1245997  0.07674543 0.00260577]
[0.8 0.  0. ]
